In [1]:
import gensim
import pyspark
import re
from pyspark import SparkContext
from pyspark.sql import SQLContext



In [2]:
sc= SparkContext()
sqlContext = SQLContext(sc)

In [3]:
text=sc.textFile('Ask0729-fixed.txt')


In [4]:
temp_var = text.map(lambda k: k.split("\\t"))
temp_var=text.map(lambda k:k.split('\t'))

In [5]:
tdf=temp_var.toDF()
tdf.head(5)

[Row(_1='No', _2='>>> [1]Contact Me Now to Make $100 Today!$LINK'),
 Row(_1='No', _2='Act now to keep your life on the go!'),
 Row(_1='No', _2='Choose between $500 and $10000 dollars with up to 5 years to repay!'),
 Row(_1='No', _2='Click above to earn today.'),
 Row(_1='No', _2='Click here to receive your first $10 today:')]

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf
def regging(x):
    x=re.sub(r'^https?:\/\/.*[\r\n]*', '$LINK', x)
    x=re.sub(r'\$\d+','$AMOUNT',x)
    return x
regged=udf(lambda x:regging(x),StringType())

In [7]:
tdf=tdf.select('_1','_2',regged('_2').alias('regged'))
sent=tdf.select('regged').collect()
sent2=list(map(lambda x:x.regged.split(),sent))
#tdf=tdf.groupby('_1').agg(tdf['regged'])

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
lis=[w for l in sent2 for w in l if not w in stop_words] 
lis[5]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/karthik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'$AMOUNT'

In [9]:
model = gensim.models.Word2Vec([lis], size=100,window=5,min_count=1)

In [104]:
#model.train(sent,total_examples=len(sent),epochs=10)
model['Click']

/home/karthik/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 4.3625417e-03, -2.2204674e-03, -1.6257000e-03, -3.4811285e-03,
        8.9125318e-04, -2.0638616e-03, -1.4706858e-03, -2.6158153e-03,
        4.7692773e-03,  2.5427230e-03, -2.9554525e-03, -3.6086626e-03,
       -1.7662031e-03,  4.5318697e-03,  3.2748391e-03, -1.0689555e-03,
       -2.0337962e-03,  1.1032386e-03,  4.8393076e-03, -4.5827916e-03,
        4.4108699e-03, -3.7101568e-03,  1.4877537e-03, -4.3797609e-03,
       -4.5222114e-03,  3.1710144e-03,  2.8827381e-03,  1.5567018e-03,
        1.9863904e-03,  3.3410832e-03, -3.2295394e-03,  2.0587244e-03,
        1.3501318e-03,  4.5899763e-03, -3.0641272e-03, -1.0878211e-03,
        2.3697277e-03, -7.3500810e-04, -2.5470469e-03,  1.3277128e-03,
       -9.6789800e-04, -3.5970109e-03,  3.3406431e-03,  3.3266165e-03,
       -5.9512752e-04, -3.3800311e-03,  3.3188069e-03,  2.1722248e-04,
        4.3159831e-04, -2.1037757e-03,  4.1742385e-03,  4.0083826e-03,
        1.7354145e-03,  1.0598892e-03,  4.2140777e-03,  3.6175007e-03,
      

In [108]:
model.wv.most_similar(positive='economy')

[('Theater', 0.35622498393058777),
 ('execute', 0.35300135612487793),
 ('heart.', 0.35039252042770386),
 ('adjustments,', 0.34089893102645874),
 ('apply', 0.3370104432106018),
 ('licensed', 0.334547221660614),
 ('PM,', 0.30412378907203674),
 ('Change', 0.30378109216690063),
 ('small.', 0.3008255064487457),
 ('quote.', 0.29812100529670715)]

In [10]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


/home/karthik/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [21]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(iter(list(word2vec.values())).__next__())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])


In [54]:
lab_prep=tdf.select('_1').collect()
print (lab)
lab= [0 for x in lab_prep if x._1=='No' or 1]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [55]:
import numpy as np
etree_w2v.fit(sent,lab)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7f9f58bc8eb8>), ('extra trees', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impur...tors=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [56]:
a=etree_w2v.predict(sent[5])

In [57]:
a[0]

0

In [50]:
lab_prep[1]._1=='No'

True